### Cell 1: Imports & Setup

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

# Enable renderer
alt.renderers.enable('default')

print("✅ Libraries imported.")

✅ Libraries imported.


### Cell 2: Data Loading & Feature Engineering

In [2]:
try:
    df = pd.read_csv('country_economics_data.csv')
except FileNotFoundError:
    print("⚠️ CSV not found. Please ensure 'country_economics_data.csv' is in the directory.")
    df = pd.DataFrame(columns=['Name', 'ID', 'Region', 'GDP', 'Population', 'Area'])

# 1. Clean Data
df_clean = df.dropna(subset=['GDP', 'Population', 'Area', 'Region', 'ID']).copy()

# 2. Feature Engineering
# GDP per Capita ($ Thousands)
df_clean['GDP_per_Capita'] = df_clean['GDP'] / df_clean['Population']

# Population Density (People per sq km)
# Pop is in Millions, Area is in sq km. So (Pop * 1,000,000) / Area
df_clean['Pop_Density'] = (df_clean['Population'] * 1_000_000) / df_clean['Area']

# Tooltip Labels
df_clean['GDP_Label'] = df_clean['GDP'].map('${:,.0f}B'.format)
df_clean['GDP_PC_Label'] = df_clean['GDP_per_Capita'].map('${:,.2f}k'.format)
df_clean['Density_Label'] = df_clean['Pop_Density'].map('{:,.0f}'.format)
df_clean['ID'] = df_clean['ID'].astype(int)

print("✅ Features created: GDP_per_Capita, Pop_Density")
print(df_clean[['Name', 'Pop_Density', 'GDP', 'GDP_per_Capita']].head())

✅ Features created: GDP_per_Capita, Pop_Density
          Name  Pop_Density  GDP  GDP_per_Capita
0  Afghanistan    54.735293   17        0.476190
1      Albania    82.092667   27       11.440678
2      Algeria    19.653690  264        5.639821
3       Angola    28.170370   80        2.277904
4    Argentina    16.929219  633       13.448056


### Cell 3: Interaction & Top 10 Logic

In [11]:
# Interaction 1: Brush for Scatter Plot (Dragging)
# This allows selecting ranges (e.g., "High Density" or "Low Wealth")
brush = alt.selection_interval()

# Interaction 2: Click for Map (Point Selection)
# Clicking a country selects its whole Region.
# This ensures the "Top 10" charts still have enough data to be useful.
map_click = alt.selection_point(fields=['Region'])

print("✅ Interactions defined: Brush (Drag) + Map Click (Region Filter).")

✅ Interactions defined: Brush (Drag) + Map Click (Region Filter).


### Cell 4: View 1 - The Symbol Map (Main View)

In [12]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

# 1. The Base Map
base = alt.Chart(world_map).mark_geoshape(
    fill='lightgray',
    stroke='white',
    strokeWidth=0.5
).project(type='equalEarth')

# 2. The Bubbles (Countries)
symbol_map = alt.Chart(df_clean).mark_circle(opacity=0.8, stroke='black', strokeWidth=0.5).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',

    # SIZE = Total Power (GDP)
    size=alt.Size(
        'GDP:Q',
        scale=alt.Scale(range=[10, 1000]),
        title='Total GDP ($ Billions)',
        legend=alt.Legend(format=".1s")
    ),

    # COLOR = Individual Wealth (GDP per Capita)
    color=alt.Color(
        'GDP_per_Capita:Q',
        scale=alt.Scale(scheme='plasma'),
        title='GDP per Capita ($k)'
    ),

    tooltip=['Name', 'GDP_Label', 'GDP_PC_Label', 'Density_Label'],

    # Interaction: Dim if NOT selected by Brush OR Map Click
    # Logic: If I click Asia, dim non-Asia. If I brush High Density, dim Low Density.
    opacity=alt.condition(brush & map_click, alt.value(0.8), alt.value(0.1))
).add_params(
    brush, map_click  # <--- FIX: Registers both interactions
).properties(
    width=900,
    height=550,
    title='Global Economy: Click Country to Filter Region'
)

map_view = base + symbol_map

map_view

alt.LayerChart(...)

### Cell 5: View 2 - The Comparison Bars (Top 10s)

In [13]:
# Helper to create a sorted Top 10 chart
def create_top_10(value_col, title, color_scheme):
    return alt.Chart(df_clean).mark_bar().encode(
        x=alt.X(
            f'{value_col}:Q',
            title=None
        ),
        y=alt.Y(
            'Name:N',
            sort='-x',
            title=None
        ),
        color=alt.Color(
            f'{value_col}:Q',
            scale=alt.Scale(scheme=color_scheme),
            legend=None
        ),
        tooltip=['Name', alt.Tooltip(f'{value_col}:Q', format=',.0f')]
    ).transform_filter(
        brush # Filter 1: Scatter Drag
    ).transform_filter(
        map_click # Filter 2: Map Click
    ).transform_window(
        rank='rank()',
        sort=[alt.SortField(value_col, order='descending')]
    ).transform_filter(
        alt.datum.rank <= 10 # Keep Top 10 of the filtered result
    ).properties(
        width=250,
        height=300,
        title=title
    )

bar_power = create_top_10('GDP', 'Top 10: Economic Power (Total GDP)', 'magma')
bar_wealth = create_top_10('GDP_per_Capita', 'Top 10: Individual Wealth', 'plasma')

comparison_view = alt.hconcat(bar_power, bar_wealth, spacing=20)

comparison_view

alt.HConcatChart(...)

### Cell 6: View 3 - Scatter Plot (Density vs Wealth)

In [14]:
scatter_density = alt.Chart(df_clean).mark_circle(size=60).encode(
    x=alt.X(
        'Pop_Density:Q',
        scale=alt.Scale(type='log', domain=[1, 20000]),
        title='People per Sq. Km (Log Scale)',
        axis=alt.Axis(values=[1, 10, 100, 1000, 10000], format=",")
    ),
    y=alt.Y(
        'GDP_per_Capita:Q',
        scale=alt.Scale(type='log', domain=[0.1, 200]),
        title='GDP per Capita ($k) (Log Scale)',
        axis=alt.Axis(values=[0.1, 1, 10, 100], format="$,")
    ),
    color=alt.condition(
        brush & map_click, # Highlight selected region + brushed area
        'Region:N',
        alt.value('lightgray')
    ),
    tooltip=['Name', 'Density_Label', 'GDP_PC_Label']
).add_params(
    brush, map_click
).transform_filter(
    map_click # Optional: Hide non-selected regions entirely? Or just dim them?
    # Let's just dim them (via Color condition above) to keep context.
).properties(
    width=520,
    height=250,
    title='Crowding vs. Cash (Density vs. Wealth)'
)

scatter_density

alt.Chart(...)

### Cell 7: Final Dashboard Assembly

In [16]:
right_panel = alt.vconcat(scatter_density, comparison_view, spacing=20)

dashboard = alt.hconcat(map_view, right_panel, spacing=20).properties(
    title={
        "text": ["Power vs. People: A Global Economic Contrast"],
        "subtitle": ["Interaction: Click Map to Select Region | Drag Scatter to Filter Density"],
        "fontSize": 24,
        "anchor": "middle"
    }
).configure_view(
    stroke=None
)

dashboard.save('power_people_dashboard_p5.html')
print("✅ Dashboard saved: 'power_people_dashboard_p5.html'")

dashboard

C:\Users\Cameron\OneDrive\Desktop\Stage 3\VisualizingEconomicCountryData\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  exec(code_obj, self.user_global_ns, self.user_ns)


✅ Dashboard saved: 'power_people_dashboard_p5.html'


alt.HConcatChart(...)